In [6]:
import numpy as np
import os

# Load images
first_folder_path = '/mnt/storage2/PanNuke/fold1'
images_path = os.path.join(first_folder_path, 'images.npy')
images = np.load(images_path)
print(images.shape)  # Check the shape of the images array


(2656, 256, 256, 3)


In [8]:
import cv2
import numpy as np
from sklearn.decomposition import PCA


In [12]:
from __future__ import annotations

# Clear logger to use tiatoolbox.logger
import logging

if logging.getLogger().hasHandlers():
    logging.getLogger().handlers.clear()

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
import skimage.color

from tiatoolbox import data, logger
from tiatoolbox.tools import stainnorm
from tiatoolbox.wsicore import wsireader

In [78]:
import os  # Module for interacting with the operating system (paths, directories).
import requests  # Library to send HTTP requests for downloading images.
import numpy as np  # Library for numerical operations on arrays.
from PIL import Image  # Module for handling image files.
from io import BytesIO  # Used to handle byte streams in memory.
from tiatoolbox.tools.stainnorm import ReinhardNormalizer  # Stain normalization tool.
import cv2  # OpenCV library for image processing.
from skimage.metrics import structural_similarity as ssim  # Function to compute SSIM.
from scipy.stats import pearsonr  # Function to compute Pearson Correlation Coefficient (PCC).

def download_image(image_url):
    """Download an image from a given URL."""
    response = requests.get(image_url)  # Send GET request to download image.
    response.raise_for_status()  # Raise an error if the request failed.
    img = Image.open(BytesIO(response.content)).convert("RGB")  # Convert response content into an RGB image.
    return np.array(img)  # Return the image as a NumPy array.

def calculate_pcc(original_image, normalized_image):
    """Calculate Pearson Correlation Coefficient (PCC) for each color channel."""
    pcc_r = pearsonr(original_image[:, :, 0].flatten(), normalized_image[:, :, 0].flatten())[0]  # PCC for red channel.
    pcc_g = pearsonr(original_image[:, :, 1].flatten(), normalized_image[:, :, 1].flatten())[0]  # PCC for green channel.
    pcc_b = pearsonr(original_image[:, :, 2].flatten(), normalized_image[:, :, 2].flatten())[0]  # PCC for blue channel.
    return (pcc_r + pcc_g + pcc_b) / 3  # Return average PCC across RGB channels.

def calculate_amce_alpha(target_image, processed_image):
    """Calculate AMCE-alpha (Eα)."""
    mean_target_alpha = np.mean(target_image[:, :, 0])  # Mean of the red channel in the target image.
    mean_processed_alpha = np.mean(processed_image[:, :, 0])  # Mean of the red channel in the processed image.
    return abs(mean_target_alpha - mean_processed_alpha)  # Return absolute difference between red channel means.

def calculate_amce_beta(target_image, processed_image):
    """Calculate AMCE-beta (Eβ)."""
    mean_target_beta = np.mean(target_image[:, :, 2])  # Mean of the blue channel in the target image.
    mean_processed_beta = np.mean(processed_image[:, :, 2])  # Mean of the blue channel in the processed image.
    return abs(mean_target_beta - mean_processed_beta)  # Return absolute difference between blue channel means.

def calculate_rse(original_image, normalized_image):
    """Calculate Relative Square Error (RSE)."""
    diff = original_image.astype(np.float32) - normalized_image.astype(np.float32)  # Compute pixel differences.
    squared_error = np.sum(diff ** 2)  # Sum of squared differences.
    original_squared_sum = np.sum(original_image.astype(np.float32) ** 2)  # Sum of squared pixel values of original.
    return squared_error / original_squared_sum if original_squared_sum != 0 else 0  # Return RSE.

def calculate_psnr(original_image, normalized_image):
    """Calculate Peak Signal-to-Noise Ratio (PSNR)."""
    mse = np.mean((original_image.astype(np.float32) - normalized_image.astype(np.float32)) ** 2)  # Mean Squared Error.
    return 10 * np.log10((255.0 ** 2) / mse) if mse != 0 else float('inf')  # Return PSNR (in decibels).

def calculate_ssim(original_image, normalized_image):
    """Calculate Structural Similarity Index (SSIM)."""
    original_image_gray = cv2.cvtColor(original_image, cv2.COLOR_RGB2GRAY)  # Convert original to grayscale.
    normalized_image_gray = cv2.cvtColor(normalized_image, cv2.COLOR_RGB2GRAY)  # Convert normalized to grayscale.
    original_image_gray = original_image_gray.astype(np.float64) / 255.0  # Normalize pixel values (original).
    normalized_image_gray = normalized_image_gray.astype(np.float64) / 255.0  # Normalize pixel values (normalized).
    return ssim(original_image_gray, normalized_image_gray, data_range=1.0)  # Return SSIM score.

def normalize_stains(input_folder, output_folder, reference_image_url):
    """Normalize stains in images from the input folder using the reference image."""
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist.

    # Download the reference image and convert to array
    target_image = download_image(reference_image_url)  # Download and convert reference image to array.

    # Initialize the Reinhard Normalizer
    stain_normalizer = ReinhardNormalizer()  # Create a Reinhard Normalizer object.
    # Fit the normalizer with the target image
    stain_normalizer.fit(target_image)  # Fit the normalizer using the reference image.

    # Process each image in the input folder
    for filename in sorted(os.listdir(input_folder)):  # Loop through each file in the input folder.
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):  # Check if the file is an image.
            source_path = os.path.join(input_folder, filename)  # Create the full source path for the image.
            source_image = Image.open(source_path).convert("RGB")  # Open and convert the image to RGB.
            sample = np.array(source_image)  # Convert image to a NumPy array.

            # Normalize the image using the Reinhard Normalizer
            normed_sample = stain_normalizer.transform(sample.copy())  # Perform stain normalization.

            # Save the normalized image
            output_image = Image.fromarray(normed_sample.astype('uint8'))  # Convert array back to an image.
            output_path = os.path.join(output_folder, filename)  # Define the output file path.
            output_image.save(output_path)  # Save the normalized image.

            # Calculate metrics for the image
            pcc_value = calculate_pcc(sample, normed_sample)  # Calculate PCC.
            amce_alpha_value = calculate_amce_alpha(target_image, normed_sample)  # Calculate AMCE-alpha.
            amce_beta_value = calculate_amce_beta(target_image, normed_sample)  # Calculate AMCE-beta.
            rse_value = calculate_rse(sample, normed_sample)  # Calculate RSE.
            psnr_value = calculate_psnr(sample, normed_sample)  # Calculate PSNR.
            ssim_value = calculate_ssim(sample, normed_sample)  # Calculate SSIM.

            # Print metrics for the current image
            print(f"Metrics for {filename}:")  # Print image filename.
            print(f"PCC: {pcc_value:.4f}")  # Print PCC value.
            print(f"AMCE-alpha (Eα): {amce_alpha_value:.4f}")  # Print AMCE-alpha value.
            print(f"AMCE-beta (Eβ): {amce_beta_value:.4f}")  # Print AMCE-beta value.
            print(f"RSE: {rse_value:.4f}")  # Print RSE value.
            print(f"PSNR: {psnr_value:.4f}")  # Print PSNR value.
            print(f"SSIM: {ssim_value:.4f}")  # Print SSIM value.
            

# Example usage
input_folder = "/mnt/storage2/PanNuke/fold1/Thyroid/images"  # Replace with your input folder path.
output_folder = "/mnt/storage2/PanNuke/folder1/Reinhard_folder/R_Testis"  # Replace with your output folder path.
reference_image_url = "https://data.kitware.com/api/v1/file/57718cc28d777f1ecd8a883c/download"  # Replace with your URL.

normalize_stains(input_folder, output_folder, reference_image_url)  # Call function to start normalization process.


Metrics for img_Testis_1_01411.png:
PCC: 0.9994
AMCE-alpha (Eα): 1.6229
AMCE-beta (Eβ): 0.1182
RSE: 0.0125
PSNR: 22.4072
SSIM: 0.9921
--------------------------------------------------
Metrics for img_Testis_1_01412.png:
PCC: 0.9995
AMCE-alpha (Eα): 1.7673
AMCE-beta (Eβ): 0.3685
RSE: 0.0061
PSNR: 26.0405
SSIM: 0.9970
--------------------------------------------------
Metrics for img_Testis_1_01413.png:
PCC: 0.9995
AMCE-alpha (Eα): 1.8912
AMCE-beta (Eβ): 0.6919
RSE: 0.0066
PSNR: 25.6047
SSIM: 0.9966
--------------------------------------------------
Metrics for img_Testis_1_01414.png:
PCC: 0.9915
AMCE-alpha (Eα): 2.8902
AMCE-beta (Eβ): 0.5055
RSE: 0.0091
PSNR: 23.9288
SSIM: 0.9565
--------------------------------------------------
Metrics for img_Testis_1_01415.png:
PCC: 0.9986
AMCE-alpha (Eα): 1.7772
AMCE-beta (Eβ): 0.3679
RSE: 0.0040
PSNR: 27.8978
SSIM: 0.9876
--------------------------------------------------
Metrics for img_Testis_1_01416.png:
PCC: 0.9965
AMCE-alpha (Eα): 1.7422
AMC

In [79]:
import os  # Provides functions to interact with the operating system, like creating directories.
import requests  # Allows making HTTP requests to download data from the internet.
import numpy as np  # Supports working with large, multi-dimensional arrays and matrices.
from PIL import Image  # Provides an Image class for opening, manipulating, and saving images.
from io import BytesIO  # Allows handling byte streams, useful for handling in-memory binary data.
from tiatoolbox.tools.stainnorm import MacenkoNormalizer  # Imports Macenko stain normalization from TIA Toolbox.
import cv2  # Provides image processing functions via OpenCV.
from skimage.metrics import structural_similarity as ssim  # Imports SSIM for image similarity comparison.
from scipy.stats import pearsonr  # Provides the Pearson correlation coefficient calculation.

# Function to download an image from a URL
def download_image(image_url):
    """Download an image from a given URL."""
    response = requests.get(image_url)  # Sends a GET request to the specified URL.
    response.raise_for_status()  # Raises an exception if the request was unsuccessful.
    img = Image.open(BytesIO(response.content)).convert("RGB")  # Opens the image in RGB mode.
    return np.array(img)  # Converts the image to a numpy array.

# Function to calculate the Pearson Correlation Coefficient for RGB channels
def calculate_pcc(original_image, normalized_image):
    """Calculate Pearson Correlation Coefficient (PCC) for each color channel."""
    # Flatten each RGB channel and calculate PCC between original and normalized images
    pcc_r = pearsonr(original_image[:, :, 0].flatten(), normalized_image[:, :, 0].flatten())[0]  # Red channel PCC
    pcc_g = pearsonr(original_image[:, :, 1].flatten(), normalized_image[:, :, 1].flatten())[0]  # Green channel PCC
    pcc_b = pearsonr(original_image[:, :, 2].flatten(), normalized_image[:, :, 2].flatten())[0]  # Blue channel PCC
    return (pcc_r + pcc_g + pcc_b) / 3  # Returns the average PCC across RGB channels.

# Function to calculate AMCE-alpha between target and processed images
def calculate_amce_alpha(target_image, processed_image):
    """Calculate AMCE-alpha (Eα)."""
    mean_target_alpha = np.mean(target_image[:, :, 0])  # Calculates the mean for the Red channel in the target image.
    mean_processed_alpha = np.mean(processed_image[:, :, 0])  # Calculates the mean for the Red channel in the processed image.
    return abs(mean_target_alpha - mean_processed_alpha)  # Returns the absolute difference.

# Function to calculate AMCE-beta between target and processed images
def calculate_amce_beta(target_image, processed_image):
    """Calculate AMCE-beta (Eβ)."""
    mean_target_beta = np.mean(target_image[:, :, 2])  # Calculates the mean for the Blue channel in the target image.
    mean_processed_beta = np.mean(processed_image[:, :, 2])  # Calculates the mean for the Blue channel in the processed image.
    return abs(mean_target_beta - mean_processed_beta)  # Returns the absolute difference.

# Function to calculate the Relative Square Error between two images
def calculate_rse(original_image, normalized_image):
    """Calculate Relative Square Error (RSE)."""
    diff = original_image.astype(np.float32) - normalized_image.astype(np.float32)  # Calculates pixel differences.
    squared_error = np.sum(diff ** 2)  # Sums the squared differences.
    original_squared_sum = np.sum(original_image.astype(np.float32) ** 2)  # Sums squares of original image values.
    return squared_error / original_squared_sum if original_squared_sum != 0 else 0  # Computes RSE if valid denominator.

# Function to calculate Peak Signal-to-Noise Ratio (PSNR) between two images
def calculate_psnr(original_image, normalized_image):
    """Calculate Peak Signal-to-Noise Ratio (PSNR)."""
    mse = np.mean((original_image.astype(np.float32) - normalized_image.astype(np.float32)) ** 2)  # Calculates MSE.
    return 10 * np.log10((255.0 ** 2) / mse) if mse != 0 else float('inf')  # Computes PSNR, handling zero MSE.

# Function to calculate Structural Similarity Index (SSIM) between two images
def calculate_ssim(original_image, normalized_image):
    """Calculate Structural Similarity Index (SSIM)."""
    original_image_gray = cv2.cvtColor(original_image, cv2.COLOR_RGB2GRAY)  # Converts original image to grayscale.
    normalized_image_gray = cv2.cvtColor(normalized_image, cv2.COLOR_RGB2GRAY)  # Converts normalized image to grayscale.
    original_image_gray = original_image_gray.astype(np.float64) / 255.0  # Normalizes pixel values to [0,1].
    normalized_image_gray = normalized_image_gray.astype(np.float64) / 255.0  # Normalizes pixel values to [0,1].
    return ssim(original_image_gray, normalized_image_gray, data_range=1.0)  # Computes SSIM between grayscale images.

# Main function to perform stain normalization on images in a directory
def normalize_stains(input_folder, output_folder, reference_image_url):
    """Normalize stains in images from the input folder using the reference image."""
    # Ensure output directory exists, creating it if necessary
    os.makedirs(output_folder, exist_ok=True)

    # Download reference image for normalization
    target_image = download_image(reference_image_url)  # Downloads and converts the reference image to array.

    # Initialize and fit the Macenko stain normalizer with the target image
    stain_normalizer = MacenkoNormalizer()  # Initializes Macenko Normalizer instance.
    stain_normalizer.fit(target_image)  # Fits the normalizer with the reference image for target stain profile.

    # Iterate over each image file in the input folder
    for filename in sorted(os.listdir(input_folder)):  # Sorts and loops over files in input folder.
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):  # Processes only image files.
            source_path = os.path.join(input_folder, filename)  # Gets the full path of the source image.
            source_image = Image.open(source_path).convert("RGB")  # Opens and converts the image to RGB mode.
            sample = np.array(source_image)  # Converts image to numpy array.

            # Normalize the image using the Macenko Normalizer
            normed_sample = stain_normalizer.transform(sample.copy())  # Applies stain normalization.

            # Save the normalized image
            output_image = Image.fromarray(normed_sample.astype('uint8'))  # Converts array back to image format.
            output_path = os.path.join(output_folder, filename)  # Constructs output path.
            output_image.save(output_path)  # Saves normalized image to output folder.

            # Calculate image quality metrics
            pcc_value = calculate_pcc(sample, normed_sample)  # Calculates PCC between original and normalized images.
            amce_alpha_value = calculate_amce_alpha(target_image, normed_sample)  # Calculates AMCE-alpha.
            amce_beta_value = calculate_amce_beta(target_image, normed_sample)  # Calculates AMCE-beta.
            rse_value = calculate_rse(sample, normed_sample)  # Calculates RSE.
            psnr_value = calculate_psnr(sample, normed_sample)  # Calculates PSNR.
            ssim_value = calculate_ssim(sample, normed_sample)  # Calculates SSIM.

            # Print the metrics for each image
            print(f"Metrics for {filename}:")
            print(f"PCC: {pcc_value:.4f}")
            print(f"AMCE-alpha (Eα): {amce_alpha_value:.4f}")
            print(f"AMCE-beta (Eβ): {amce_beta_value:.4f}")
            print(f"RSE: {rse_value:.4f}")
            print(f"PSNR: {psnr_value:.4f}")
            print(f"SSIM: {ssim_value:.4f}")
            

# Example usage with folder paths and reference image URL (customize with your paths and URL)
input_folder = "/mnt/storage2/PanNuke/fold1/Thyroid/images"  # Define the input folder containing images.
output_folder = "/mnt/storage2/PanNuke/folder1/Mancenko_folder/M_Testis"  # Define the folder to save normalized images.
reference_image_url = "https://data.kitware.com/api/v1/file/57718cc28d777f1ecd8a883c/download"  # URL of the reference image.

# Run the normalization process
normalize_stains(input_folder, output_folder, reference_image_url)  # Calls the main function to process images.


Metrics for img_Testis_1_01411.png:
PCC: 0.9896
AMCE-alpha (Eα): 19.8370
AMCE-beta (Eβ): 19.5612
RSE: 0.0414
PSNR: 17.1948
SSIM: 0.9424
--------------------------------------------------
Metrics for img_Testis_1_01412.png:
PCC: 0.9878
AMCE-alpha (Eα): 31.1976
AMCE-beta (Eβ): 33.7690
RSE: 0.0503
PSNR: 16.9003
SSIM: 0.9337
--------------------------------------------------
Metrics for img_Testis_1_01413.png:
PCC: 0.9929
AMCE-alpha (Eα): 14.4370
AMCE-beta (Eβ): 17.9189
RSE: 0.0207
PSNR: 20.6396
SSIM: 0.9829
--------------------------------------------------
Metrics for img_Testis_1_01414.png:
PCC: 0.9860
AMCE-alpha (Eα): 16.9654
AMCE-beta (Eβ): 14.6332
RSE: 0.0261
PSNR: 19.3690
SSIM: 0.9477
--------------------------------------------------
Metrics for img_Testis_1_01415.png:
PCC: 0.9854
AMCE-alpha (Eα): 22.2486
AMCE-beta (Eβ): 29.6420
RSE: 0.0393
PSNR: 17.9209
SSIM: 0.9504
--------------------------------------------------
Metrics for img_Testis_1_01416.png:
PCC: 0.9876
AMCE-alpha (Eα): 

In [80]:
# Import necessary libraries for working with images, downloading data, and performing various metrics
import os
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from tiatoolbox.tools.stainnorm import VahadaneNormalizer  # Updated to Vahadane for stain normalization
import cv2
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr

# Function to download an image from a URL
def download_image(image_url):
    """Download an image from a given URL."""
    response = requests.get(image_url)  # Send HTTP GET request to the URL
    response.raise_for_status()  # Raise error if the response has issues
    img = Image.open(BytesIO(response.content)).convert("RGB")  # Convert the response content into an RGB image
    return np.array(img)  # Return the image as a NumPy array

# Function to calculate Pearson Correlation Coefficient (PCC) for each color channel
def calculate_pcc(original_image, normalized_image):
    """Calculate Pearson Correlation Coefficient (PCC) for each color channel."""
    # Calculate PCC for the red, green, and blue channels individually
    pcc_r = pearsonr(original_image[:, :, 0].flatten(), normalized_image[:, :, 0].flatten())[0]
    pcc_g = pearsonr(original_image[:, :, 1].flatten(), normalized_image[:, :, 1].flatten())[0]
    pcc_b = pearsonr(original_image[:, :, 2].flatten(), normalized_image[:, :, 2].flatten())[0]
    # Return the average PCC over RGB channels
    return (pcc_r + pcc_g + pcc_b) / 3  

# Function to calculate the AMCE-alpha metric (Eα)
def calculate_amce_alpha(target_image, processed_image):
    """Calculate AMCE-alpha (Eα)."""
    # Compute mean of the alpha channel (assumed to be the Red channel here) in both images
    mean_target_alpha = np.mean(target_image[:, :, 0])  
    mean_processed_alpha = np.mean(processed_image[:, :, 0])
    # Return the absolute difference as AMCE-alpha
    return abs(mean_target_alpha - mean_processed_alpha)

# Function to calculate the AMCE-beta metric (Eβ)
def calculate_amce_beta(target_image, processed_image):
    """Calculate AMCE-beta (Eβ)."""
    # Compute mean of the beta channel (assumed to be the Blue channel here) in both images
    mean_target_beta = np.mean(target_image[:, :, 2])  
    mean_processed_beta = np.mean(processed_image[:, :, 2])
    # Return the absolute difference as AMCE-beta
    return abs(mean_target_beta - mean_processed_beta)

# Function to calculate Relative Square Error (RSE)
def calculate_rse(original_image, normalized_image):
    """Calculate Relative Square Error (RSE)."""
    # Calculate squared difference between original and normalized images
    diff = original_image.astype(np.float32) - normalized_image.astype(np.float32)
    squared_error = np.sum(diff ** 2)
    # Compute the squared sum of the original image
    original_squared_sum = np.sum(original_image.astype(np.float32) ** 2)
    # Return RSE if original squared sum is non-zero
    return squared_error / original_squared_sum if original_squared_sum != 0 else 0

# Function to calculate Peak Signal-to-Noise Ratio (PSNR)
def calculate_psnr(original_image, normalized_image):
    """Calculate Peak Signal-to-Noise Ratio (PSNR)."""
    # Calculate mean squared error between original and normalized images
    mse = np.mean((original_image.astype(np.float32) - normalized_image.astype(np.float32)) ** 2)
    # Return PSNR value using formula
    return 10 * np.log10((255.0 ** 2) / mse) if mse != 0 else float('inf')

# Function to calculate Structural Similarity Index (SSIM)
def calculate_ssim(original_image, normalized_image):
    """Calculate Structural Similarity Index (SSIM)."""
    # Convert both images to grayscale for SSIM calculation
    original_image_gray = cv2.cvtColor(original_image, cv2.COLOR_RGB2GRAY)
    normalized_image_gray = cv2.cvtColor(normalized_image, cv2.COLOR_RGB2GRAY)
    # Normalize the grayscale images to [0, 1]
    original_image_gray = original_image_gray.astype(np.float64) / 255.0
    normalized_image_gray = normalized_image_gray.astype(np.float64) / 255.0
    # Calculate and return SSIM
    return ssim(original_image_gray, normalized_image_gray, data_range=1.0)

# Function to normalize stains for images in a folder using a reference image
def normalize_stains(input_folder, output_folder, reference_image_url):
    """Normalize stains in images from the input folder using the reference image."""
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Download the reference image and convert to array
    target_image = download_image(reference_image_url)

    # Initialize the Vahadane Normalizer
    stain_normalizer = VahadaneNormalizer()  # Updated to Vahadane normalization
    # Fit the normalizer with the target image
    stain_normalizer.fit(target_image)

    # Loop through each image in the input folder
    for filename in sorted(os.listdir(input_folder)):
        # Process only files with specific image extensions
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):
            source_path = os.path.join(input_folder, filename)  # Get path to the source image
            source_image = Image.open(source_path).convert("RGB")  # Open and convert image to RGB
            sample = np.array(source_image)  # Convert image to NumPy array

            # Normalize the image using the Vahadane Normalizer
            normed_sample = stain_normalizer.transform(sample.copy())

            # Save the normalized image
            output_image = Image.fromarray(normed_sample.astype('uint8'))
            output_path = os.path.join(output_folder, filename)  # Define output path for normalized image
            output_image.save(output_path)  # Save image in the output folder

            # Calculate metrics for the image
            pcc_value = calculate_pcc(sample, normed_sample)
            amce_alpha_value = calculate_amce_alpha(target_image, normed_sample)
            amce_beta_value = calculate_amce_beta(target_image, normed_sample)
            rse_value = calculate_rse(sample, normed_sample)
            psnr_value = calculate_psnr(sample, normed_sample)
            ssim_value = calculate_ssim(sample, normed_sample)

            # Print calculated metrics for the current image
            print(f"Metrics for {filename}:")
            print(f"PCC: {pcc_value:.4f}")
            print(f"AMCE-alpha (Eα): {amce_alpha_value:.4f}")
            print(f"AMCE-beta (Eβ): {amce_beta_value:.4f}")
            print(f"RSE: {rse_value:.4f}")
            print(f"PSNR: {psnr_value:.4f}")
            print(f"SSIM: {ssim_value:.4f}")
            

# Example usage
input_folder = "/mnt/storage2/PanNuke/fold1/Thyroid/images"  # Specify your input folder path here
output_folder = "/mnt/storage2/PanNuke/folder1/vahadane_folder/V_Testis"  # Specify your output folder path here
reference_image_url = "https://data.kitware.com/api/v1/file/57718cc28d777f1ecd8a883c/download"  # Reference image URL

normalize_stains(input_folder, output_folder, reference_image_url)


Metrics for img_Testis_1_01411.png:
PCC: 0.9793
AMCE-alpha (Eα): 4.0926
AMCE-beta (Eβ): 2.7989
RSE: 0.0181
PSNR: 20.7866
SSIM: 0.9729
--------------------------------------------------
Metrics for img_Testis_1_01412.png:
PCC: 0.9863
AMCE-alpha (Eα): 14.2178
AMCE-beta (Eβ): 13.2068
RSE: 0.0170
PSNR: 21.6051
SSIM: 0.9784
--------------------------------------------------
Metrics for img_Testis_1_01413.png:
PCC: 0.9872
AMCE-alpha (Eα): 0.4162
AMCE-beta (Eβ): 2.0854
RSE: 0.0100
PSNR: 23.7945
SSIM: 0.9936
--------------------------------------------------
Metrics for img_Testis_1_01414.png:
PCC: 0.9900
AMCE-alpha (Eα): 15.4319
AMCE-beta (Eβ): 18.8666
RSE: 0.0336
PSNR: 18.2616
SSIM: 0.9573
--------------------------------------------------
Metrics for img_Testis_1_01415.png:
PCC: 0.9928
AMCE-alpha (Eα): 11.4515
AMCE-beta (Eβ): 13.6054
RSE: 0.0148
PSNR: 22.1530
SSIM: 0.9821
--------------------------------------------------
Metrics for img_Testis_1_01416.png:
PCC: 0.9916
AMCE-alpha (Eα): 0.01

In [81]:
# Import necessary libraries for image processing, stain normalization, and metrics calculation
import os  # For interacting with the operating system
import requests  # To handle HTTP requests and download images
import numpy as np  # For numerical operations on image data
from PIL import Image  # To handle image processing
from io import BytesIO  # For in-memory binary I/O operations
from tiatoolbox.tools.stainnorm import RuifrokNormalizer  # Import Ruifrok stain normalization tool from tiatoolbox
import cv2  # OpenCV library for image processing
from skimage.metrics import structural_similarity as ssim  # For calculating structural similarity between images
from scipy.stats import pearsonr  # For calculating Pearson correlation coefficient

def download_image(image_url):
    """Download an image from a given URL."""
    response = requests.get(image_url)  # Send HTTP request to download the image
    response.raise_for_status()  # Raise an error if the request fails
    img = Image.open(BytesIO(response.content)).convert("RGB")  # Open the image and convert it to RGB format
    return np.array(img)  # Convert the image to a numpy array and return it

def calculate_pcc(original_image, normalized_image):
    """Calculate Pearson Correlation Coefficient (PCC) for each color channel."""
    pcc_r = pearsonr(original_image[:, :, 0].flatten(), normalized_image[:, :, 0].flatten())[0]  # PCC for red channel
    pcc_g = pearsonr(original_image[:, :, 1].flatten(), normalized_image[:, :, 1].flatten())[0]  # PCC for green channel
    pcc_b = pearsonr(original_image[:, :, 2].flatten(), normalized_image[:, :, 2].flatten())[0]  # PCC for blue channel
    return (pcc_r + pcc_g + pcc_b) / 3  # Return the average PCC across all channels

def calculate_amce_alpha(target_image, processed_image):
    """Calculate AMCE-alpha (Eα) by measuring absolute difference in mean red values."""
    mean_target_alpha = np.mean(target_image[:, :, 0])  # Calculate mean for the red channel in target image
    mean_processed_alpha = np.mean(processed_image[:, :, 0])  # Calculate mean for red channel in processed image
    return abs(mean_target_alpha - mean_processed_alpha)  # Return absolute difference in means

def calculate_amce_beta(target_image, processed_image):
    """Calculate AMCE-beta (Eβ) by measuring absolute difference in mean blue values."""
    mean_target_beta = np.mean(target_image[:, :, 2])  # Calculate mean for the blue channel in target image
    mean_processed_beta = np.mean(processed_image[:, :, 2])  # Calculate mean for blue channel in processed image
    return abs(mean_target_beta - mean_processed_beta)  # Return absolute difference in means

def calculate_rse(original_image, normalized_image):
    """Calculate Relative Square Error (RSE) between original and normalized images."""
    diff = original_image.astype(np.float32) - normalized_image.astype(np.float32)  # Difference between images
    squared_error = np.sum(diff ** 2)  # Sum of squared errors across all pixels
    original_squared_sum = np.sum(original_image.astype(np.float32) ** 2)  # Total squared value of original image
    return squared_error / original_squared_sum if original_squared_sum != 0 else 0  # Return RSE

def calculate_psnr(original_image, normalized_image):
    """Calculate Peak Signal-to-Noise Ratio (PSNR) between original and normalized images."""
    mse = np.mean((original_image.astype(np.float32) - normalized_image.astype(np.float32)) ** 2)  # Mean squared error
    return 10 * np.log10((255.0 ** 2) / mse) if mse != 0 else float('inf')  # Return PSNR or infinity if MSE is zero

def calculate_ssim(original_image, normalized_image):
    """Calculate Structural Similarity Index (SSIM) between grayscale versions of images."""
    original_image_gray = cv2.cvtColor(original_image, cv2.COLOR_RGB2GRAY)  # Convert original image to grayscale
    normalized_image_gray = cv2.cvtColor(normalized_image, cv2.COLOR_RGB2GRAY)  # Convert normalized image to grayscale
    original_image_gray = original_image_gray.astype(np.float64) / 255.0  # Normalize pixel values to [0, 1]
    normalized_image_gray = normalized_image_gray.astype(np.float64) / 255.0  # Normalize pixel values to [0, 1]
    return ssim(original_image_gray, normalized_image_gray, data_range=1.0)  # Calculate SSIM

def normalize_stains(input_folder, output_folder, reference_image_url):
    """Normalize stains in images from the input folder using the reference image."""
    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    target_image = download_image(reference_image_url)  # Download and convert the reference image to an array

    stain_normalizer = RuifrokNormalizer()  # Initialize the Ruifrok stain normalizer
    stain_normalizer.fit(target_image)  # Fit the normalizer with the reference image

    for filename in sorted(os.listdir(input_folder)):  # Loop through sorted files in the input folder
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):  # Process only image files
            source_path = os.path.join(input_folder, filename)  # Full path to the image file
            source_image = Image.open(source_path).convert("RGB")  # Open and convert the source image to RGB
            sample = np.array(source_image)  # Convert the image to a numpy array

            normed_sample = stain_normalizer.transform(sample.copy())  # Normalize the image using Ruifrok Normalizer

            output_image = Image.fromarray(normed_sample.astype('uint8'))  # Convert normalized array back to an image
            output_path = os.path.join(output_folder, filename)  # Path to save the normalized image
            output_image.save(output_path)  # Save the normalized image

            # Calculate metrics to assess quality of stain normalization
            pcc_value = calculate_pcc(sample, normed_sample)  # Calculate Pearson Correlation Coefficient (PCC)
            amce_alpha_value = calculate_amce_alpha(target_image, normed_sample)  # Calculate AMCE-alpha
            amce_beta_value = calculate_amce_beta(target_image, normed_sample)  # Calculate AMCE-beta
            rse_value = calculate_rse(sample, normed_sample)  # Calculate Relative Square Error (RSE)
            psnr_value = calculate_psnr(sample, normed_sample)  # Calculate Peak Signal-to-Noise Ratio (PSNR)
            ssim_value = calculate_ssim(sample, normed_sample)  # Calculate Structural Similarity Index (SSIM)

            # Display the calculated metrics for each image
            print(f"Metrics for {filename}:")
            print(f"PCC: {pcc_value:.4f}")
            print(f"AMCE-alpha (Eα): {amce_alpha_value:.4f}")
            print(f"AMCE-beta (Eβ): {amce_beta_value:.4f}")
            print(f"RSE: {rse_value:.4f}")
            print(f"PSNR: {psnr_value:.4f}")
            print(f"SSIM: {ssim_value:.4f}")
            

# Example usage
input_folder = "/mnt/storage2/PanNuke/fold1/Thyroid/images"  # Path to folder with input images
output_folder = "/mnt/storage2/PanNuke/folder1/Ruifork_folder/ru_Testis"  # Path to folder for saving normalized images
reference_image_url = "https://data.kitware.com/api/v1/file/57718cc28d777f1ecd8a883c/download"  # URL for reference image

normalize_stains(input_folder, output_folder, reference_image_url)  # Execute the stain normalization process


Metrics for img_Testis_1_01411.png:
PCC: 0.9878
AMCE-alpha (Eα): 31.3334
AMCE-beta (Eβ): 44.7301
RSE: 0.0223
PSNR: 19.8742
SSIM: 0.9668
--------------------------------------------------
Metrics for img_Testis_1_01412.png:
PCC: 0.9820
AMCE-alpha (Eα): 40.6950
AMCE-beta (Eβ): 37.7968
RSE: 0.0290
PSNR: 19.2900
SSIM: 0.9705
--------------------------------------------------
Metrics for img_Testis_1_01413.png:
PCC: 0.9840
AMCE-alpha (Eα): 26.0973
AMCE-beta (Eβ): 47.4595
RSE: 0.0231
PSNR: 20.1668
SSIM: 0.9809
--------------------------------------------------
Metrics for img_Testis_1_01414.png:
PCC: 0.9519
AMCE-alpha (Eα): 31.6590
AMCE-beta (Eβ): 44.5382
RSE: 0.0372
PSNR: 17.8234
SSIM: 0.9765
--------------------------------------------------
Metrics for img_Testis_1_01415.png:
PCC: 0.9671
AMCE-alpha (Eα): 32.3548
AMCE-beta (Eβ): 42.4486
RSE: 0.0347
PSNR: 18.4655
SSIM: 0.9726
--------------------------------------------------
Metrics for img_Testis_1_01416.png:
PCC: 0.9726
AMCE-alpha (Eα): 